In [560]:
import json
import cobrakbase

In [24]:
token = "64XQ7SABQILQWSEW3CQKZXJA63DXZBGH"
kbase = cobrakbase.KBaseAPI(token)
kbase_dev = cobrakbase.KBaseAPI(token, dev=True)

In [389]:
#kbase_dev.save_object('iML1515', 'filipeliu:narrative_1580723870549', 'KBaseFBA.FBAModel', model1)
#kbase_dev.save_object('ecoli.kbase', 'filipeliu:narrative_1580723870549', 'KBaseFBA.FBAModel', model2)

In [534]:
model3 = kbase_dev.get_object('test_propagated_model', 'chenry:narrative_1504151898593')

In [ ]:
with open('../data/html/data/model1.json', 'w') as fh:
    fh.write(json.dumps(model1))
with open('../data/html/data/model2.json', 'w') as fh:
    fh.write(json.dumps(model2))

In [538]:
models = {
    'iML1515' : fbamodel1,
    'KBase' : fbamodel2,
    'model3' : fbamodel3,
}

In [535]:
fbamodel3 = cobrakbase.core.KBaseFBAModel(model3)
b = cobrakbase.core.converters.KBaseFBAModelToCobraBuilder(fbamodel3)
cobra_model3 = b.build()

In [561]:
cobra.io.read_sbml_model('/Users/fliu/Downloads/iML1515.kb.xml')

CobraSBMLError: Something went wrong reading the SBML model. Most likely the SBML model is not valid. Please check that your model is valid using the `cobra.io.sbml.validate_sbml_model` function or via the online validator at http://sbml.org/validator .
	`(model, errors) = validate_sbml_model(filename)`
If the model is valid and cannot be read please open an issue at https://github.com/opencobra/cobrapy/issues .

In [413]:
kbase_dev.ws_client.get_workspace_info({'id' :8248 })

[8248,
 'chenry:narrative_1504151898593',
 'chenry',
 '2020-02-10T20:01:50+0000',
 253,
 'a',
 'r',
 'unlocked',
 {'narrative_nice_name': 'fba_tools_test_narrative',
  'is_temporary': 'false',
  'narrative': '1'}]

In [416]:
os = kbase_dev.list_objects('chenry:narrative_1504151898593')
for o in os:
    oid = o[1]
    if oid == 'test_expression_matrix':
        break
o

[170,
 'test_expression_matrix',
 'KBaseFeatureValues.ExpressionMatrix-1.0',
 '2017-09-08T16:30:31+0000',
 2,
 'jjeffryes',
 8248,
 'chenry:narrative_1504151898593',
 '5a100ee2b834c6d17ad83902451382c6',
 1095819,
 None]

In [447]:
ref = kbase_dev.get_object_info_from_ref('8248/232/1')

In [448]:
ref.workspace_id

'chenry:narrative_1504151898593'

In [449]:
attribute_mapping = kbase_dev.get_object(ref.id, ref.workspace_id)

In [456]:
attribute_mapping['attributes'][-1]

{'attribute': 'seed_id',
 'attribute_ont_id': 'Custom:Term',
 'attribute_ont_ref': 'KbaseOntologies/Custom',
 'source': ''}

In [460]:
attribute_mapping['instances']['2-methylmaleate']

['', '564', 'C5H6O4', '130.0266087', 'HNEGQIOMVPPMNR-IHWYPQMZSA-N', 'cpd01502']

In [429]:
exodata = kbase_dev.get_object('exodata', 'chenry:narrative_1504151898593')

In [440]:
exodata.keys()

dict_keys(['biochemistry_ref', 'col_attributemapping_ref', 'col_mapping', 'data', 'description', 'row_attributemapping_ref', 'row_mapping', 'scale'])

In [446]:
exodata['row_attributemapping_ref']

'8248/232/1'

In [514]:
#exodata['data']['col_ids']

In [510]:
#exodata['data']['row_ids']
#ChemicalAbundanceMatrix

In [546]:
ref_id = test_expression_matrix['genome_ref'] #'8248/9/1'
ref = kbase_dev.get_object_info_from_ref(ref_id)


In [551]:
model3['modelreactions'][0]['modelReactionProteins']

[{'complex_ref': '~/template/complexes/name/cpx01779',
  'modelReactionProteinSubunits': [{'feature_refs': ['~/genome/features/id/SAMA_RS00835'],
    'note': '',
    'optionalSubunit': 0,
    'role': 'ftr00745',
    'triggering': 1}],
  'note': '',
  'source': ''}]

In [417]:
test_expression_matrix = kbase_dev.get_object('test_expression_matrix', 'chenry:narrative_1504151898593')

In [485]:
test_expression_matrix.keys()

dict_keys(['data', 'feature_mapping', 'genome_ref', 'scale', 'type'])

In [487]:
test_expression_matrix['scale']

'1.0'

In [532]:
exo_data = get_chemical_abundance_data(exodata, 
                                       'ZMMG_Z_mobilis_ZMO0748_cysK', 
                                       attribute_mapping)
exo_data = get_all_chemical_abundance_data(exodata, attribute_mapping)

map_decorators = dump_chemical_abundance_datasets('model3', exo_data, map_decorators)

In [ ]:
params = {
    'workspace_id': 37534, 
    'map_ids': ['dfsdfsdf'], 
    'grid_x': '1', 'grid_y': '1', 
    'model_objects': [
        {'object_ids': ['37534/91/1'], 'model_id': '37534/90/3'}, 
        {'object_ids': ['37534/92/1'], 'model_id': '37534/89/1'}], 
    'report_height': '800', 
    'workspace_name': 'filipeliu:narrative_1580723870549', 
    'model_ids': ['37534/90/3', '37534/89/1']
}

In [509]:
expr_data = get_all_expression_data(test_expression_matrix)


map_decorators = dump_expression_datasets('model3', expr_data, map_decorators)
with open('../data/html/data/map_decorators.json', 'w') as fh:
    fh.write(json.dumps(map_decorators))

In [562]:
#expr_data

In [409]:
metabolite_conc = {
    'cpd01775_c0' : 23.0,
    'cpd01777_c0' : 20.0
}

with open('../data/html/data/model1_cpd_conc.json', 'w') as fh:
    fh.write(json.dumps(metabolite_conc))

metabolite_conc = {
    'cpd01775_c0' : 20.0,
    'cpd01777_c0' : 19
}

with open('../data/html/data/model2_cpd_conc.json', 'w') as fh:
    fh.write(json.dumps(metabolite_conc))

In [3]:
model1 = kbase.get_object('iML1515.kb', 'filipeliu:narrative_1504192868437')

In [4]:
model2 = kbase.get_object('GCF_000005845.2.mdl', 'filipeliu:narrative_1504192868437')

In [120]:
fbamodel1 = cobrakbase.core.KBaseFBAModel(model1)
fbamodel2 = cobrakbase.core.KBaseFBAModel(model2)

In [508]:
map_decorators = {
    "iML1515": {
        "reaction": {
            "FBA" : [
                {"id" : "Glucose", "path" : "model1_flux_glucose.json"},
                {"id" : "Etho", "path" : "model1_flux_etho.json"},
                {"id" : "Ac", "path" : "model1_flux_ac.json"}
            ]
        },
        "compound": {
            "concentation" : [
                {"id" : "cpd_conc", "path" : "model1_cpd_conc.json"},
            ]
        },
        "gene": {
            "expression" : [
                {"id" : "gene_exp", "path" : "model1_expre.json"},
            ]
        }
    }, 
    "KBase": {
        "reaction" : {
            "FBA" : [
                {"id" : "Glucose", "path" : "model2_flux_glucose.json"}
            ]
        },
        "compound": {
            "concentation" : [
                {"id" : "cpd_conc", "path" : "model2_cpd_conc.json"},
            ]
        },
        "gene": {
            "expression" : [
                {"id" : "gene_exp", "path" : "model2_expre.json"},
            ]
        }
    }
}
with open('../data/html/data/map_decorators.json', 'w') as fh:
    fh.write(json.dumps(map_decorators))

In [543]:
with open('../data/html/data/map_decorators.json', 'w') as fh:
    fh.write(json.dumps(map_decorators))

In [539]:
for model_id in models:
    b = cobrakbase.core.converters.KBaseFBAModelToCobraBuilder(models[model_id])
    cobra_model = b.build()
    cobra_json = json.loads(cobra.io.to_json(cobra_model))
    for m in cobra_json['metabolites']:
        m['id'] += '@' + model_id
    for r in cobra_json['reactions']:
        r['id'] += '@' + model_id
        r['metabolites'] = dict(map(lambda x : (x + '@' + model_id, r['metabolites'][x]), r['metabolites']))
    with open('../data/html/data/models/' + model_id + '.json', 'w') as fh:
        fh.write(json.dumps(cobra_json))

In [183]:
b = cobrakbase.core.converters.KBaseFBAModelToCobraBuilder(fbamodel1)
cobra_model1 = b.build()

In [190]:
media = cobrakbase.core.KBaseBiochemMedia(kbase.get_object('Carbon-D-Glucose', 'filipeliu:narrative_1504192868437'))

In [191]:
b = cobrakbase.core.converters.KBaseFBAModelToCobraBuilder(fbamodel2)
cobra_model2 = b.with_media(media).build()

In [203]:
solution = cobra_model2.optimize()
flux_map = dict(map(lambda x : (x.id, x.flux), cobra_model2.reactions))
with open('../data/html/data/model2_flux_glucose.json', 'w') as fh:
    fh.write(json.dumps(flux_map))

1

In [384]:
flux_map2 = {}
for r in fbamodel1.reactions:
    oid = r.get_original_id()
    if oid == None:
        print(r.id)
    else:
        oid = oid[2:]
        if oid in flux_map:
            flux_map2[r.id] = flux_map[oid]
with open('../data/html/data/model1_flux_etho.json', 'w') as fh:
    fh.write(json.dumps(flux_map2))

DM_cpd02701_c0
DM_cpd01027_c0
DM_cpd15380_c0
DM_cpd04122_c0
DM_cpd01042_c0
DM_cpd15127_c0


In [383]:
cobra_model0 = cobra.io.read_sbml_model('../../../../data/sbml/iML1515/source.xml')
#cobra_model0.summary()
#cobra_model0.objective = cobra_model0.reactions.BIOMASS_Ec_iML1515_WT_75p37M
cobra_model0.objective = cobra_model0.reactions.BIOMASS_Ec_iML1515_core_75p37M
#cobra_model0.reactions.EX_ac_e.lower_bound = -30
cobra_model0.reactions.EX_glc__D_e.lower_bound = 0
cobra_model0.reactions.EX_etoh_e.lower_bound = -30
solution = cobra_model0.optimize()
print(solution)
flux_map = dict(map(lambda x : (x.id, x.flux), cobra_model0.reactions))
#cobra_model0.medium

<Solution 1.165 at 0x7f9a3e51ea50>


In [8]:
ws_dev = 'filipeliu:narrative_1580723870549'

In [10]:
import escher
import modelseed_escher

In [11]:
escher_seed = modelseed_escher.EscherManager(escher)

In [13]:
escher_seed.list_datasets()

{'BIOS',
 'Bacillus subtilis',
 'Escherichia coli',
 'Homo sapiens',
 'KEGG',
 'ModelSEED',
 'Neurospora crassa',
 'Oscarichia coli',
 'Pichia pastoris',
 'Pseudomonas putida KT2440',
 'Saccharomyces cerevisiae',
 'Synechococcus',
 'consensus',
 'raw'}

In [587]:
map_data= None
with open('/Users/fliu/Downloads/new_map (4).json', 'r') as fh:
    map_data = json.loads(fh.read())
    
odata = adapt_map_to_kbase(modelseed_escher.core.EscherMap(map_data), "example_map")
kbase_dev.save_object("example_map", ws_dev, "KBaseFBA.EscherMap", odata)

[[107,
  'example_map',
  'KBaseFBA.EscherMap-3.0',
  '2020-02-18T21:48:37+0000',
  6,
  'filipeliu',
  37534,
  'filipeliu:narrative_1580723870549',
  '8f6b080d6549ca6e06fabed9d0e3bad3',
  9611,
  {}]]

In [586]:
def adapt_map_to_kbase(map_data, map_id):
    map_data.escher_data['map_name'] = map_id
    if not 'authors' in map_data.escher_data:
        map_data.escher_data['authors'] = []
    for rxn_uid in map_data.escher_graph['reactions']:
        rxn_node = map_data.escher_graph['reactions'][rxn_uid]
        rxn_node['reversibility'] = 1 if rxn_node['reversibility'] else 0
    
        #TEMP HACK FIX OBJECT SPEC
        for seg_uid in rxn_node['segments']:
            seg = rxn_node['segments'][seg_uid]
            if seg['b1'] == None:
                #seg['b1'] = {'x' : 0, 'y' : 0}
                del seg['b1']
            if seg['b2'] == None:
                #seg['b2'] = {'x' : 0, 'y' : 0}
                del seg['b2']
    for node_uid in map_data.escher_graph['nodes']:
        node = map_data.escher_graph['nodes'][node_uid]
        if 'node_is_primary' in node:
            node['node_is_primary'] = 1 if node['node_is_primary'] else 0
        if not 'node_is_primary' in node:
            node['node_is_primary'] = 0
    kbase_escher = {
        "metadata" : map_data.escher_data,
        "layout" : map_data.escher_graph
    }
    o_id = map_id.replace(',', ' ') \
                 .replace(')', ' ') \
                 .replace('(', ' ') \
                 .replace(' ', '_') \
                 .replace('__', '_')
    
    return kbase_escher

In [575]:
for m in escher_seed.list_maps('ModelSEED'):
    model_id, map_id = m.split('.')
    map_data = escher_seed.get_map('ModelSEED', model_id, map_id)
    map_data.escher_data['map_name'] = map_id
    if not 'authors' in map_data.escher_data:
        map_data.escher_data['authors'] = []
    for rxn_uid in map_data.escher_graph['reactions']:
        rxn_node = map_data.escher_graph['reactions'][rxn_uid]
        rxn_node['reversibility'] = 1 if rxn_node['reversibility'] else 0
    
        #TEMP HACK FIX OBJECT SPEC
        for seg_uid in rxn_node['segments']:
            seg = rxn_node['segments'][seg_uid]
            if seg['b1'] == None:
                seg['b1'] = {'x' : 0, 'y' : 0}
            if seg['b2'] == None:
                seg['b2'] = {'x' : 0, 'y' : 0}
    for node_uid in map_data.escher_graph['nodes']:
        node = map_data.escher_graph['nodes'][node_uid]
        if 'node_is_primary' in node:
            node['node_is_primary'] = 1 if node['node_is_primary'] else 0
        if not 'node_is_primary' in node:
            node['node_is_primary'] = 0
    kbase_escher = {
        "metadata" : map_data.escher_data,
        "layout" : map_data.escher_graph
    }
    o_id = map_id.replace(',', ' ') \
                 .replace(')', ' ') \
                 .replace('(', ' ') \
                 .replace(' ', '_') \
                 .replace('__', '_')
    #print(map_data.escher_data['map_name'], o_id)
    kbase_dev.save_object(o_id, ws_dev, "KBaseFBA.EscherMap", kbase_escher)

KeyboardInterrupt: 

In [79]:
for m in escher_seed.list_maps('ModelSEED'):
    model_id, map_id = m.split('.')
    map_data = escher_seed.get_map('ModelSEED', model_id, map_id)
    print(map_id)
    break

Xyloglucan Metabolism


In [81]:
map_data = escher_seed.get_map('ModelSEED', model_id, 'FBA Checked TCA Cycle')

In [127]:

scan_content(map_data, fbamodel2, None)

{'map_cpd': 33, 'cpd_in_map_count': 33, 'map_rxn': 19, 'rxn_in_map_count': 16}

In [103]:
m = fbamodel1.reactions[0]

In [107]:
m.get_gpr()

[{'b0241', 'b0929', 'b1377', 'b2215'}]

In [115]:
m.data['reaction_ref'].split('/')[-1].split('_')[0]

'rxn00000'

In [304]:
map_list = {}
for m in escher_seed.list_maps('ModelSEED'):
    model_id, map_id = m.split('.')
    map_data = escher_seed.get_map('ModelSEED', model_id, map_id)
    map_list[map_id] = {}
    for model_id in models:
        map_list[map_id][model_id] = {}
        map_list[map_id][model_id] = scan_content(map_data, models[model_id], None)
with open('../data/html/data/map_list.json', 'w') as fh:
    fh.write(json.dumps(map_list))

In [163]:
grid = modelseed_escher.EscherGrid()

In [565]:
em_list = []
map_assembly = [
    {'map_id' : 'FBA Checked TCA Cycle', 'cmp_id' : 'c0', 'model_id' : 'iML1515'},
    {'map_id' : 'FBA Checked TCA Cycle', 'cmp_id' : 'c0', 'model_id' : 'KBase'},
    {'map_id' : 'FBA Checked TCA Cycle', 'cmp_id' : 'c0', 'model_id' : 'model3'},
    {'map_id' : 'Histidine Biosynthesis', 'cmp_id' : 'c0', 'model_id' : 'iML1515'},
    {'map_id' : 'Histidine Biosynthesis', 'cmp_id' : 'c0', 'model_id' : 'KBase'},
    {'map_id' : 'Histidine Biosynthesis', 'cmp_id' : 'c0', 'model_id' : 'model3'},
]
for grid_block_data in map_assembly:
    map_id = grid_block_data['map_id']
    cmp_id = grid_block_data['cmp_id']
    model_id = grid_block_data['model_id']
    fbamodel = models[model_id]
    em = escher_seed.get_map('ModelSEED', 'ModelSEED', map_id)
    em = adapt_map_to_model(em, cmp_id, model_id, fbamodel)
    
    em_list.append(em)
grid_map = grid.build(em_list, (3, 2))

In [402]:
def adapt_map_to_model(em, cmp_id, suffix, fbamodel):
    em = em.clone()
    #adapt map to compartment
    move_to_compartment(cmp_id, em)
    
    map_cpd_set = set()
    map_rxn_set = set()
    for node_uid in em.nodes:
        node = em.escher_graph['nodes'][node_uid]
        if node['node_type'] == 'metabolite':
            map_cpd_set.add(node['bigg_id'])
    for rxn_uid in em.escher_graph['reactions']:
        rxn_node = em.escher_graph['reactions'][rxn_uid]
        map_rxn_set.add(rxn_node['bigg_id'])
    
    model_cpds = set(map(lambda x : x.id, fbamodel.metabolites))
    model_rxns = set(map(lambda x : x.id, fbamodel.reactions))
    
    map_cpd_delete = map_cpd_set - model_cpds
    map_rxn_delete = map_rxn_set - model_rxns
    
    #remove metabolites / reactions
    em.delete_reactions(map_rxn_delete)
    em.delete_metabolites(map_cpd_delete)
    
    #add suffix
    cpd_remap = dict(map(lambda x : (x, x + '@' + suffix), map_cpd_set))
    rxn_remap = dict(map(lambda x : (x, x + '@' + suffix), map_rxn_set))
    em.swap_ids(cpd_remap, rxn_remap)
    #update rev
    return em

In [300]:
em = escher_seed.get_map('ModelSEED', 'ModelSEED', map_id)
adapt_map_to_model(em, cmp_id, model_id, fbamodel)

In [ ]:
    def translate_to_model(self, escher_model_id, escher_map_id, sbml_id, target_cmp, cpd_mapping, rxn_mapping):
        em = self.escher_manager.get_map('ModelSEED', escher_model_id, escher_map_id)
        em.add_uid_to_reaction_metabolites()

        move_to_compartment(target_cmp, em)

        map_compound_remap, unmaped_cpd, node_uid_cmp = remap_map_compounds(em, 
                                                                            cpd_mapping, lambda x : x.split('_'))

        logger.warning('unmaped_cpd: %d', len(unmaped_cpd))

        map_reaction_remap, unmaped_rxn = remap_map_reactions(em, rxn_mapping, map_compound_remap, self.cpd_match_exclude,
                                                              self.lambda_get_model_reaction(sbml_id),
                                                              lambda x : x, lambda x : x[:-2])

        logger.warning('unmaped_rxn: %d', len(unmaped_rxn))

        print(sbml_id, map_reaction_remap)
        
        cpd_remap = {}
        rxn_remap = {}
        for k in map_compound_remap:
            cpd_remap[k] = list(map_compound_remap[k])[0] + '@' + sbml_id
        for k in map_reaction_remap:
            rxn_remap[k] = list(map_reaction_remap[k])[0] + '@' + sbml_id
        #print(sbml_id, rxn_remap)
        em.swap_ids(cpd_remap, rxn_remap)
        em.delete_reactions(unmaped_rxn)
        em.delete_metabolites(unmaped_cpd)
        return em

In [566]:
with open('../data/html/data/escher_map.json', 'w') as fh:
    fh.write(json.dumps(grid_map.escher_map))

In [557]:
option_list = []
for m in escher_seed.list_maps('ModelSEED'):
    model_id, map_id = m.split('.')
    #print(map_id)
    option_list.append({
        'id' : map_id,
        'display' : map_id,
        'ui_name' : map_id,
        'value' : map_id
    })

In [ ]:
escher_seed.list_maps()